In [1]:
import vertexai
from vertexai.language_models import TextEmbeddingInput
from vertexai.language_models import TextEmbeddingModel
import numpy as np
from dotenv import load_dotenv
import os
import chromadb

In [2]:
load_dotenv()
Project = os.getenv('GOOGLE_PROJECT')
Region = os.getenv('REGION_GOOGLE')
vertexai.init(project=Project, location=Region)

In [3]:
import google.generativeai as genai
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

c:\Users\wyatt\ActualDocumentation\Hackathons\HackAI\Backend\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
model = TextEmbeddingModel("textembedding-gecko@003")

In [ ]:
import getClubData
import google.generativeai as genai
from google.api_core import retry
index, title, category, missionDescripts, presidentName, email, pictureUrl = getClubData.getData()
embedStrings = getClubData.getEmbeddingString(index, title, category, missionDescripts)

print(embedStrings)
retry_policy = {"retry": retry.Retry(predicate=retry.if_transient_error)}
clubEmbedding = genai.embed_content(
            model="models/text-embedding-004",
            content=embedStrings,
            task_type='RETRIEVAL_DOCUMENT',
            request_options=retry_policy
)

import json
with open("./data/club_embeddings.json", "w") as f:
    json.dump(clubEmbedding, f)

print("Embeddings saved in JSON format!")
#clubEmbedding = model.get_embeddings(
#    texts=embedStrings,
#    task_type="RETRIEVAL_DOCUMENT"
#)
# clubEmbedding = TextEmbeddingInput(embedStrings, 'RETRIVAL_DOCUMENT')
# print(clubEmbedding)
# clubEmbedding = [embedding.values for embedding in model.get_embeddings(clubEmbedding)][0]

Index: 2
Title: Aaja Nachele
Category: Art and Music, Cultural
Mission Description: We are organization that hosts a yearly Bollywood fusion competition hosting teams from all around the US, partaking in a national dance circuit known as DDN. We facilitate and set up the competition here in Dallas and ensure we have everything ready and prepped for the next year's event.
President's Name: Reeba Biji, Arsh Thakker, Harika Pappu
Email: hxp200023@utdallas.edu, aht200000@utdallas.edu, rmb200007@utdallas.edu
Picture URL: /sites/StudentOrganizationCenterSP/Shared%20Documents/Apps/Microsoft%20Forms/Organization%20Registration%20Form/Cover%20Image/new%20logo%20af_Dinesh%20Kanneboina.png
----------------------------------------
Index: 3
Title: AAPG & SEG Student Chapter
Category: Academic Interests, Educational/Departmental
Mission Description: AAPG & SEG Student Chapter brings together a group of like-minded individuals who share a common interest in geology geophysics. Providing opportunities

In [2]:
#configuration = chromadb.config.Settings(chroma_db_impl="sqlite", persist_directory="chromaClubs")
#print(chromadb.__version__)
import json
import getClubData
import chromadb
index, title, category, missionDescripts, presidentName, email, pictureUrl = getClubData.getData()
embedStrings = getClubData.getEmbeddingString(index, title, category, missionDescripts)
with open("./data/club_embeddings.json", "r") as f:
    data = json.load(f)
clubEmbedding = data.get("embedding")
chromaClient = chromadb.PersistentClient()
chromaClient.delete_collection("embeddedClubs")
embedClubsCollection = chromaClient.get_or_create_collection(name="embeddedClubs")
embedClubsCollection.add(
    documents=embedStrings,
    ids=[str(i) for i in index],
    embeddings=clubEmbedding
)
print(embedClubsCollection.count())

Index: 2
Title: Aaja Nachele
Category: Art and Music, Cultural
Mission Description: We are organization that hosts a yearly Bollywood fusion competition hosting teams from all around the US, partaking in a national dance circuit known as DDN. We facilitate and set up the competition here in Dallas and ensure we have everything ready and prepped for the next year's event.
President's Name: Reeba Biji, Arsh Thakker, Harika Pappu
Email: hxp200023@utdallas.edu, aht200000@utdallas.edu, rmb200007@utdallas.edu
Picture URL: /sites/StudentOrganizationCenterSP/Shared%20Documents/Apps/Microsoft%20Forms/Organization%20Registration%20Form/Cover%20Image/new%20logo%20af_Dinesh%20Kanneboina.png
----------------------------------------
Index: 3
Title: AAPG & SEG Student Chapter
Category: Academic Interests, Educational/Departmental
Mission Description: AAPG & SEG Student Chapter brings together a group of like-minded individuals who share a common interest in geology geophysics. Providing opportunities

In [34]:
testInput = ["Machine Learning and Cyber Security"]
testInputEmbedding = genai.embed_content(
            model="models/text-embedding-004",
            content=testInput,
            task_type='RETRIEVAL_QUERY',
            request_options=retry_policy
)
results = embedClubsCollection.query(
    query_embeddings=testInputEmbedding["embedding"],
    n_results=15
)
print(results)
documents = results.get("documents", [])
print(documents)
print(j for j in (i for i in documents))
for doc_list in documents:
    for doc in doc_list: 
        print(doc)

{'ids': [['404', '123', '111', '463', '197', '33', '32', '91', '162', '37', '52', '34', '177', '60', '287']], 'embeddings': None, 'documents': [['{Title: The Cyber Defense and Response Unit\n,Category: Educational/Departmental, Special Interest\n,Mission Description: The mission of this organization is to foster academic exploration and build a robust community of defense-oriented cyber professionals. We are committed to training the next generation of cyber defenders by offering comprehensive education, training, and hands-on experiences in key areas such as digital forensics, incident response, and cyber defense. Our goal is to equip our members with the skills and knowledge necessary to protect individuals and organizations, while promoting ethical and responsible practices in the field of cybersecurity.}', '{Title: Cybersecurity Club\n,Category: Academic Interests\n,Mission Description: We host talks from individuals in the cybersecurity industry to show students what a career in c